IR PRACTICAL 3: BAYESIAN NETWORK FOR HEART DISEASE

Step 1: Import Required Libraries

In [5]:
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

Step 2: Define the Network Structure

In [6]:
model = DiscreteBayesianNetwork([
    ('Age', 'BloodPressure'),
    ('Age', 'Cholesterol'),
    ('Age', 'HeartDisease'),
    ('BloodPressure', 'HeartDisease'),
    ('Cholesterol', 'HeartDisease'),
    ('HeartDisease', 'ChestPain')
])

Step 3: Define Conditional Probability Tables (CPTs)

In [7]:
# Age
cpd_age = TabularCPD(variable='Age', variable_card=2,
                     values=[[0.6], [0.4]],
                     state_names={'Age': ['Young', 'Old']})

# Blood Pressure | Age
cpd_bp = TabularCPD(variable='BloodPressure', variable_card=2,
                    values=[[0.8, 0.3], [0.2, 0.7]],
                    evidence=['Age'], evidence_card=[2],
                    state_names={'BloodPressure': ['Normal', 'High'], 'Age': ['Young', 'Old']})

# Cholesterol | Age
cpd_chol = TabularCPD(variable='Cholesterol', variable_card=2,
                      values=[[0.85, 0.4], [0.15, 0.6]],
                      evidence=['Age'], evidence_card=[2],
                      state_names={'Cholesterol': ['Normal', 'High'], 'Age': ['Young', 'Old']})
# Heart Disease | Age, BloodPressure, Cholesterol
cpd_hd = TabularCPD(variable='HeartDisease', variable_card=2,
                    values=[
                        [0.95, 0.8, 0.7, 0.4, 0.8, 0.5, 0.4, 0.1],
                        [0.05, 0.2, 0.3, 0.6, 0.2, 0.5, 0.6, 0.9]
                    ],
                    evidence=['Age', 'BloodPressure', 'Cholesterol'],
                    evidence_card=[2, 2, 2],
                    state_names={
                        'HeartDisease': ['No', 'Yes'],
                        'Age': ['Young', 'Old'],
                        'BloodPressure': ['Normal', 'High'],
                        'Cholesterol': ['Normal', 'High']
                    })
# Chest Pain | Heart Disease
cpd_cp = TabularCPD(variable='ChestPain', variable_card=2,
                    values=[[0.9, 0.3], [0.1, 0.7]],
                    evidence=['HeartDisease'], evidence_card=[2],
                    state_names={'ChestPain': ['No', 'Yes'], 'HeartDisease': ['No', 'Yes']})


Step 4: Add CPTs to the Model

In [8]:
model.add_cpds(cpd_age, cpd_bp, cpd_chol, cpd_hd, cpd_cp)

In [9]:
# Validate the model
assert model.check_model()
print(" Bayesian Network successfully constructed.")

 Bayesian Network successfully constructed.


Step 5: Perform Inference

In [10]:
infer = VariableElimination(model)

# Example Query: Probability of Heart Disease given Age=Old, BP=High, Chol=High
query_result = infer.query(variables=['HeartDisease'],
                           evidence={'Age': 'Old', 'BloodPressure': 'High', 'Cholesterol': 'High'})
print("\n Inference Result:")
print(query_result)


 Inference Result:
+-------------------+---------------------+
| HeartDisease      |   phi(HeartDisease) |
+===================+=====================+
| HeartDisease(No)  |              0.1000 |
+-------------------+---------------------+
| HeartDisease(Yes) |              0.9000 |
+-------------------+---------------------+
